# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kapa'a,22.0752,-159.3190,19.71,95,0,4.12,US,1738867846
1,1,ushuaia,-54.8000,-68.3000,8.81,71,75,12.86,AR,1738867847
2,2,xinqing,48.2333,129.5000,-17.18,98,100,1.53,CN,1738867849
3,3,edinburgh of the seven seas,-37.0676,-12.3116,17.03,91,47,9.92,SH,1738867850
4,4,port-aux-francais,-49.3500,70.2167,6.46,84,100,8.78,TF,1738867851


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    "Lng", "Lat", 
    geo=True, 
    tiles="OSM",
    size="Humidity", 
    color="Humidity", 
    colormap="coolwarm",
    alpha=0.75
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [17]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp_min = 20   
ideal_temp_max = 30   
ideal_humidity_max = 60  
ideal_wind_max = 10   
ideal_cloudiness_max = 20  

# Filter cities based on ideal conditions
ideal_cities_df = city_data_df.loc[
    (city_data_df["Max Temp"] >= ideal_temp_min) & 
    (city_data_df["Max Temp"] <= ideal_temp_max) & 
    (city_data_df["Humidity"] <= ideal_humidity_max) & 
    (city_data_df["Wind Speed"] <= ideal_wind_max) & 
    (city_data_df["Cloudiness"] <= ideal_cloudiness_max)
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,mbaiki,3.8678,17.9892,26.72,23,8,0.49,CF,1738867854
29,29,ras al khaimah city,25.7895,55.9432,23.12,56,0,9.26,AE,1738867878
47,47,ciudad lazaro cardenas,17.9583,-102.2000,28.72,56,0,4.33,MX,1738867899
132,132,crane,31.3974,-102.3501,25.30,13,0,2.61,US,1738867995
165,165,atbara,17.7022,33.9864,24.33,14,0,9.55,SD,1738868030


### Step 3: Create a new DataFrame called `hotel_df`.

In [23]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,kapa'a,US,22.0752,-159.3190,95,
1,ushuaia,AR,-54.8000,-68.3000,71,
2,xinqing,CN,48.2333,129.5000,98,
3,edinburgh of the seven seas,SH,-37.0676,-12.3116,91,
4,port-aux-francais,TF,-49.3500,70.2167,84,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [29]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "filter": "", 
    "bias": "",  
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"  
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kapa'a - nearest hotel: Pono Kai Resort
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
xinqing - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
port-aux-francais - nearest hotel: Keravel
puerto natales - nearest hotel: Dorotea Patagonia Hostel
the pas - nearest hotel: Wescana Inn
mbaiki - nearest hotel: No hotel found
severo-yeniseyskiy - nearest hotel: Актолик
bella vista - nearest hotel: 21c Museum Hotel Bentonville
qaqortoq - nearest hotel: Hotel Qaqortoq
sisimiut - nearest hotel: Hotel SØMA
adamstown - nearest hotel: No hotel found
waitangi - nearest hotel: Hotel Chathams
zabaykal'sk - nearest hotel: Гостиница Русь
nordborg - nearest hotel: Hotel Nørherredhus
bredasdorp - nearest hotel: Victoria Hotel
muisne - nearest hotel: Hostal Té de Menta
didwana - nearest hotel: No hotel found
freetown - nearest hotel: Formal Leona hotel
vorgashor - nearest hotel: София
sungai penuh - nearest hotel: Hotel Matahari
hadibu 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,kapa'a,US,22.0752,-159.3190,95,Pono Kai Resort
1,ushuaia,AR,-54.8000,-68.3000,71,Apart Hotel Aires del Beagle
2,xinqing,CN,48.2333,129.5000,98,No hotel found
3,edinburgh of the seven seas,SH,-37.0676,-12.3116,91,No hotel found
4,port-aux-francais,TF,-49.3500,70.2167,84,Keravel
...,...,...,...,...,...,...
580,hazard,US,37.2495,-83.1932,85,Hampton Inn & Suites
581,'ain el hadjar,DZ,34.7585,0.1453,74,Hôtel Forsane
582,santa cruz,BO,-17.8000,-63.1667,58,Aviador Hotel Boutique
583,lagos,NG,6.5833,3.7500,80,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [31]:
%%capture --no-display

hotel_map = hotel_df.hvplot.points(
    "Lng",  
    "Lat",  
    geo=True,  
    tiles="OSM",  # OpenStreetMap 
    color="Humidity",  # Color by Humidity
    size=100,  # Adjust marker size
    frame_width=700,  
    frame_height=500, 
    hover_cols=["City", "Country", "Hotel Name"]  
)
    
# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)